# Example of Ask/Tell interface with DEAP 

In [1]:
import random
import array
import numpy

from deap import base, benchmarks, creator, tools, algorithms
from deap import cma

In [2]:
import numpy as np
from brian2 import *
from brian2.equations.equations import (DIFFERENTIAL_EQUATION, Equations,
                                        SingleEquation, PARAMETER)
from brian2.input import TimedArray
from brian2 import NeuronGroup, StateMonitor, store, restore, run, defaultclock, second, Quantity
from brian2.stateupdaters.base import StateUpdateMethod

### Input

In [3]:
input_traces = zeros((10,1))*volt
for i in range(1):
    input_traces[1:,i]=i*10*mV

In [4]:
# Create target current traces
output_traces = 10*nS*input_traces

In [5]:
input = input_traces
output = output_traces

In [6]:
params = np.array([
 [ 1.80869973e-08,  2.50218013e-02],
 [ 1.88373085e-08,  9.89559934e-02], 
 [ 1.88373085e-08,  9.89559934e-02], 
])

### Setup The Model for Optimization

In [7]:
input_var = 'v'
output_var = 'I'

parameter_names = {'g', 'E'}
method = ('linear', 'exponential_euler', 'euler')
t_start = 0*second
popsize, _ = np.shape(params)
dt = 0.1 *ms
defaultclock.dt = dt

In [8]:
model = Equations('''
I = g*(v-E) : amp
g : siemens (constant)
E : volt (constant)
''')

In [9]:
state_update_code = StateUpdateMethod.apply_stateupdater(model, {}, method=method)

INFO       No numerical integration method specified, using method 'linear' (took 0.00s). [brian2.stateupdaters.base.method_choice]


### Required Model Operations

In [10]:
Nsteps, Ntraces = input_traces.shape
# N = popsize * len(parameter_names)
N = popsize
duration = Nsteps*dt

In [11]:
model_without_diffeq = Equations([eq for eq in model.ordered
                                      if eq.type != DIFFERENTIAL_EQUATION])
    
# Add a parameter for each differential equation
diffeq_params = Equations([SingleEquation(PARAMETER, varname, model.dimensions[varname])
                           for varname in model.diff_eq_names])

# Our new model:
model = model_without_diffeq + diffeq_params

# Replace input variable by TimedArray
input_traces = TimedArray(input, dt = dt)

In [12]:
input_unit = input.dim
model = model + Equations(input_var + '= input_var(t,i % Ntraces) : '+ "% s" % repr(input_unit))

# Add criterion with TimedArray
output_traces = TimedArray(output, dt = dt)
error_unit = output.dim**2
model = model + Equations('total_error : %s' % repr(error_unit))

In [13]:
neurons = NeuronGroup(Ntraces*N, model, method = method)
neurons.namespace['input_var'] = input_traces
neurons.namespace['output_var'] = output_traces
neurons.namespace['t_start'] = t_start
neurons.namespace['Ntraces'] = Ntraces

#### Record error  
additional differential equation calculating the error

In [14]:
neurons.run_regularly('total_error +=  (' + output_var + '-output_var(t,i % Ntraces))**2 * int(t>=t_start)',
                      when='end')

# Add the code doing the numerical integration
neurons.run_regularly(state_update_code, when='groups')

# store the state of the network
store()

In [15]:
def parameters_dict(params):
    d = dict()
    for name, value in zip(parameter_names, params.T):
        d[name] = value
            
    return d

In [16]:
def calc_error(params):
    print(params)
    popsize, _ = np.shape(params)
    N = popsize

#     neurons = NeuronGroup(Ntraces*N, model, method = method)
    neurons = NeuronGroup(N, model, method = method)
    neurons.namespace['input_var'] = input_traces
    neurons.namespace['output_var'] = output_traces
    neurons.namespace['t_start'] = t_start
    neurons.namespace['Ntraces'] = Ntraces

    # Record error
    neurons.run_regularly('total_error +=  (' + output_var + '-output_var(t,i % Ntraces))**2 * int(t>=t_start)',
                          when='end')

    # Add the code doing the numerical integration
    neurons.run_regularly(state_update_code, when='groups')

    d = parameters_dict(params)
    neurons.set_states(d, units=False)
    run(duration, namespace = {})

    e = neurons.total_error/int((duration-t_start)/defaultclock.dt)
    e = mean(e.reshape((N,Ntraces)),axis=1)
    
    return array(e)

In [17]:
neurons

NeuronGroup(clock=Clock(dt=100. * usecond, name='defaultclock'), when=start, order=0, name='neurongroup')

### Ask and Tel with calc_error Function

In [18]:
start_scope()

In [19]:
model = Equations('''
I = g*(v-E) : amp
g : siemens (constant)
E : volt (constant)
''')

In [20]:
state_update_code = StateUpdateMethod.apply_stateupdater(model, {}, method=method)

In [21]:
model_without_diffeq = Equations([eq for eq in model.ordered
                                      if eq.type != DIFFERENTIAL_EQUATION])
    
# Add a parameter for each differential equation
diffeq_params = Equations([SingleEquation(PARAMETER, varname, model.dimensions[varname])
                           for varname in model.diff_eq_names])

# Our new model:
model = model_without_diffeq + diffeq_params

# Replace input variable by TimedArray
input_traces = TimedArray(input, dt = dt)

In [22]:
input_unit = input.dim
model = model + Equations(input_var + '= input_var(t,i % Ntraces) : '+ "% s" % repr(input_unit))

# Add criterion with TimedArray
output_traces = TimedArray(output, dt = dt)
error_unit = output.dim**2
model = model + Equations('total_error : %s' % repr(error_unit))

### set up DEAP

In [23]:
NDIM = 2
IND_SIZE = 5

creator.create("FitnessMin", base.Fitness, weights=(-1.0, -1.0))
creator.create("Individual", list, fitness=creator.FitnessMin)

strategy = cma.Strategy(centroid=[2.0]*NDIM, sigma=1.0)

toolbox = base.Toolbox()
toolbox.register("attr_float", np.random.uniform, -2, 2)
toolbox.register("individual", tools.initRepeat, creator.Individual,
                 toolbox.attr_float, n=IND_SIZE)

toolbox.register("generate", strategy.generate, creator.__dict__["Individual"])
toolbox.register("update", strategy.update)

In [24]:
hof = tools.HallOfFame(1)
stats = tools.Statistics(lambda ind: ind.fitness.values)

In [25]:
stats = None
ngen = 1

logbook = tools.Logbook()
logbook.header = ['gen', 'nevals'] + (stats.fields if stats else [])

for gen in range(ngen):
    # Generate a new population
    population = toolbox.generate()
    print(population, '\n')

[[1.6380106144194795, 0.34135222519397956], [1.5030782070779367, 1.5316639874589073], [2.0222407693309328, 2.5588964459828945], [1.421425696679829, 2.7914288187794525], [0.9282562628011248, 3.212375997997304], [2.0654229637090347, 1.6245011223160644]] 



In [26]:
population

[[1.6380106144194795, 0.34135222519397956],
 [1.5030782070779367, 1.5316639874589073],
 [2.0222407693309328, 2.5588964459828945],
 [1.421425696679829, 2.7914288187794525],
 [0.9282562628011248, 3.212375997997304],
 [2.0654229637090347, 1.6245011223160644]]

In [27]:
fitnesses = calc_error(np.array(population))

for ind, fit in zip(population, fitnesses):
    print(ind, fit)
    ind.fitness.values = (fit,)

[[1.63801061 0.34135223]
 [1.50307821 1.53166399]
 [2.02224077 2.55889645]
 [1.4214257  2.79142882]
 [0.92825626 3.212376  ]
 [2.06542296 1.62450112]]
[1.6380106144194795, 0.34135222519397956] 0.3126359383631386
[1.5030782070779367, 1.5316639874589073] 5.300174383991945
[2.0222407693309328, 2.5588964459828945] 26.77756891396502
[1.421425696679829, 2.7914288187794525] 15.743505510517574
[0.9282562628011248, 3.212375997997304] 8.89177614712651
[2.0654229637090347, 1.6245011223160644] 11.25791678014446


In [28]:
hof.update(population)

In [29]:
toolbox.update(population)

In [30]:
record = stats.compile(population) if stats is not None else {}
logbook.record(gen=gen, nevals=len(population), **record)

print(logbook.stream)

gen	nevals
0  	6     


In [31]:
hof[0]

[1.6380106144194795, 0.34135222519397956]

In [32]:
hof[0].fitness.values[0]

0.3126359383631386

In [33]:
print("Best individual is ", hof[0], hof[0].fitness.values[0])

Best individual is  [1.6380106144194795, 0.34135222519397956] 0.3126359383631386


### Working Bounds?

In [ ]:
NDIM = 2
IND_SIZE = 5

creator.create("FitnessMin", base.Fitness, weights=(-1.0, -1.0))
creator.create("Individual", list, fitness=creator.FitnessMin)

In [ ]:
strategy = cma.Strategy(centroid=[2.0]*NDIM, sigma=1.0)

In [43]:
cma.StrategyMultiObjective?

Init signature: cma.StrategyMultiObjective(population, sigma, **params)
Docstring:     
Multiobjective CMA-ES strategy based on the paper [Voss2010]_. It
is used similarly as the standard CMA-ES strategy with a generate-update
scheme.

:param population: An initial population of individual.
:param sigma: The initial step size of the complete system.
:param mu: The number of parents to use in the evolution. When not
           provided it defaults to the length of *population*. (optional)
:param lambda_: The number of offspring to produce at each generation.
                (optional, defaults to 1)
:param indicator: The indicator function to use. (optional, default to
                  :func:`~deap.tools.hypervolume`)

Other parameters can be provided as described in the next table

+----------------+---------------------------+----------------------------+
| Parameter      | Default                   | Details                    |
+================+===========================+========

In [ ]:
toolbox = base.Toolbox()
toolbox.register("attr_float", np.random.uniform, -2, 2)
toolbox.register("individual", tools.initRepeat, creator.Individual,
                 toolbox.attr_float, n=IND_SIZE)

toolbox.register("generate", strategy.generate, creator.__dict__["Individual"])
toolbox.register("update", strategy.update)